人类使用文本，计算机使用字节序列。——Esther Nam和Travis Fischer

本章主要讨论Unicode字符串、二进制序列，以及在二者之间转换时使用的编码。

## 4.1 字符问题


字符：可以理解为Unicode字符。
* 码位：字符的标识，可以理解为序号。
* 字符的具体表述，取决于所用的编码。

> Python3的`str`对象和Python2的`Unicode`对象的元素是Unicode字符，Python2的`str`对象获取的是原始字节序列，但专门的二进制序列提供的功能，有些是Py2的`str`类型不具有的。

> `编码`：把码位（str对象对应码位）转换为字节序列（bytes对象）。`解码`：把字节序列转换为码位。

In [31]:
# 编码和解码
s = 'café'
len(s)

4

In [32]:
b = s.encode('utf8') # str对象变为bytes对象，bytes字面量以b开头
b

b'caf\xc3\xa9'

In [33]:
len(b) # 在UTF-8中，é编码为两个字节，即16位

5

In [34]:
b.decode('utf8') # bytes对象变为str对象

'café'

## 4.2 字节概要
本节介绍Python内置的两种基本的二进制序列类型：
* Python 3引入的不可变的`bytes`类型
* Python 2.6添加的可变`bytearray`类型
> Python2.6的`bytes`类型只不过是`str`的别称。

`bytes`或`bytearray`对象的各个元素是介于0～255（含）之间的整数（即8位，一个字节）。

In [43]:
# 包含5个字节的bytes和bytesarray对象
cafe = bytes('café', encoding='utf8')
cafe # 'caf'是可打印字符，直接显示，而‘é’不是

b'caf\xc3\xa9'

In [36]:
cafe[0]

99

In [37]:
cafe[:1] # 二进制序列的切片始终是同一类型的二进制序列，对比之下string[0] == string[:1]为True

b'c'

In [40]:
cafe_arr = bytearray(cafe) # bytearray对象没有字面量句法，而是以bytearray()和字节序列字面量参数的形式显示
cafe_arr

bytearray(b'caf\xc3\xa9')

In [42]:
cafe_arr[-1:] # 二进制序列的切片始终是同一类型的二进制序列

bytearray(b'\xa9')

二进制序列虽然是整数序列，但从其字面量表示法可以看出，内含ASCII文本，比如cafe对应b'caf\xc3\xa9'。

二进制序列___字面量表示法___中各个字节的值可能会用以下三种方式显示：
* 可打印ASCII范围内的字节（从空格到～），使用ASCII字符本身;
* 制表符、换行符、回车符和\对应的字节，使用转义序列\t,\n,\r和\\；
* 其他字节的值，使用16进制转义序列。

`str`类型的大部分方法都支持`bytes`和`bytearray`类型。
* 可用：`endswith`，`replace`，`strip`，`translate`和`upper`等；编译自二进制序列的正则表达式。
* 不可用：除了格式化方法（`format`和`format_map`）和几个处理Unicode数据的方法，比如`casefold`，`isdecimal`，`isidentifier`，`isnumeric`，`isprintable`和`encode`）。

二进制序列有而`str`没有的方法：`fromhex`，解析十六进制数字对，构建二进制序列。

In [97]:
bytes.fromhex('314BCEA9')
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

`bytes`或`bytearray`实例的构造方法可以传入以下参数：

In [114]:
import array
from random import random

a = bytes('hello', encoding='utf8') # 一个str对象和一个encoding关键字参数
b = bytes([2, 9, 255, 187]) # 一个可迭代对象，提供0-255之间的数值
c = bytes(5) # 一个整数，使用空字节创建对应长度的二进制序列
# 一个实现了缓冲协议的对象，比如bytes，bytearray，memoryview，array.array
d = bytes(array.array('h', range(5))) 

In [109]:
a

b'hello'

In [110]:
b

b'\x02\t\xff\xbb'

In [111]:
c

b'\x00\x00\x00\x00\x00'

In [115]:
d

b'\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00'

In [118]:
array.array('h', range(5))

array('h', [0, 1, 2, 3, 4])

使用缓冲类对象创建`bytes`或`bytearray`时，始终复制源对象中的字节序列，而`memoryview`对象允许在二进制数据结构之间共享内存。
### 结构体和内存视图
`struct`模块：提供函数解决字节序列和数据之间的转换，即字节序列转换成不同类型字段组成的元组，或者相反。该模块能处理`bytes`，`bytearray`和`memoryview`。

`memoryview`对象的切片是一个新的`memoryview`对象，而不会复制字节序列。

In [129]:
import struct
fmt = '<3s3sHH'
with open('cat.gif', 'rb') as fp:
    img = fp.read()
    img_memv = memoryview(img)
img[:10]

b'GIF89a\x90\x01\x90\x01'

In [130]:
header = img_memv[:10]
bytes(header)

b'GIF89a\x90\x01\x90\x01'

In [131]:
struct.unpack(fmt, header) # 使用memoryview和struct查看一个gif图像的宽度和高度

(b'GIF', b'89a', 400, 400)

In [132]:
del header
del img_memv

## 4.3 基本的编解码器
Python自带超100种编解码器(codec, encoder/decoder)，用于文本和字节序列互相转换。每个`codec`有一个名称，比如`'utf_8'`，且常有别名，比如`'utf8'`，`'utf-8'`和`'U8'`。这些名称可以传给`open()`, `str.encode()`, `bytes.decode()`等函数作为`encoding`参数。

In [138]:
# 使用3个编解码器编码字符串El Niño
for codec in ['latin_1', 'utf_8', 'utf_16']: # latin-1即扩展ASCII，256个字符
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## 4.4 了解编解码问题
* `UnicodeEncodeError`：把字符串转换成二进制序列时
* `UnicodeDecodeError`：把二进制序列转换成字符串时
* `SyntaxError`：如果源码的编码与预期不符

### 4.4.1 处理`UnicodeEncodeError`
多数非`Unicode`编解码器只能处理`Unicode`字符的一小部分子集。在`encode`时，如果目标编码中没有定义某个字符，会抛出`UnicodeEncodeError`错误。
> 编解码器的错误处理方式是可拓展的，可以为`errors`参数注册额外的字符串，方法是把一个名称和一个错误处理函数传给`codecs.register_error`。

In [139]:
# 编码成字节序列：成功和错误处理
city = 'São Paulo'
city.encode('utf_8')

b'S\xc3\xa3o Paulo'

In [140]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [141]:
city.encode('iso8859_1') # latin_1

b'S\xe3o Paulo'

In [143]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [144]:
city.encode('cp437', errors='ignore') # IBM PC最初的字符集，包含框图符号，与后来出现的latin1不兼容

b'So Paulo'

In [145]:
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [149]:
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#227;o Paulo'

### 4.4.2 处理`UnicodeDecodeError`
1. 如果字节序列没有对应的ASCII字符或UTF-8/UTF-16字符，会抛出`UnicodeDecodeError`；
2. 一些老式8位编码——`cp1252`, `iso8859_1`和`koi8_r`等——能解码任何字节序列流而不报错。

> 乱码字符称为鬼符(gremlin)或mojibake（“变形文本的“的日文）。

In [150]:
octets = b'Montr\xe9al' # Latin1编码的
octets.decode('latin1')

'Montréal'

In [151]:
print(octets.decode('cp1252'))
print(octets.decode('iso8859_7')) # 编码希腊文的
print(octets.decode('koi8_r')) # 编码俄文的

Montréal
Montrιal
MontrИal


In [152]:
octets.decode('utf8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [155]:
octets.decode('utf8', errors='replace') # \xe9被替换成了U+FFFD，这是官方指定的替换字符

'Montr�al'

### 4.4.3 使用预期之外的编码加载模块时抛出的`SyntaxError`
Python3默认使用UTF-8编码源码，Python2则是ASCII编码。如果加载的.py模块中存在非UTF-8字符，且没有事先说明编码，会报错。

GNU/LINUX和OS X系统大都使用UTF-8。

可以在文件开头声明:
> \# coding: cp1252

源码中能不能使用非ASCII字符？源码的目的是便于目标群体阅读和编辑，而不是“所有人”。所以特定情况下，允许使用非ASCII字符，比如当地语言文字。

### 4.4.4 如何找出字节序列的编码？
不能。除非有人告诉你。

某些字节流包含大于127的字节值，我们可以确信其不是ASCII编码，但无法100\%确定二进制文件的编码是ASCII或UTF-8。

统一字符编码侦测包：Chardet，是一个Python库，也提供了命令行工具。

In [157]:
import chardet

In [165]:
with open('./floats.txt', 'rb') as fb: # 如果打开方式不是b，则报错
    print(chardet.detect(fb.read()))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [166]:
with open('./chapter1_python数据模型.ipynb', 'rb') as fb: # 如果打开方式不是b，则报错
    print(chardet.detect(fb.read()))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


二进制序列编码文本通常不会明确指明自己的编码，但UTF-8格式可以在文本内容的开头添加一个字节序标记。

### 4.4.5 BOM：有用的鬼符
BOM - 字节序标记`byte-order mark`。

字节序分大字节序（big endian）和小字节序（little endian），后者指各个码位的最低有效字节在前，所以字节序只影响一个字符占多个字节的编码.

UTF-8不需要BOM，因为它只用一个字节，无所谓高位低位，其U+FEFF字符是一个三字节序列：b'\xef\xbb\xbf'（不过Python不会因为该开头便自动假定文件使用的是UTF-8编码）。

UTF-16编码有两个变种，UTF-16LE和UTF-16BE，为区分需要在文本前加上特殊的不可见字符ZERO WIDTH NO-BREAK SPACE (U+FEFF)，在小字节序中为`b'\xff\xfe'`（十进制数255，254），这就是BOM。

根据标准，如果文件使用UTF-16编码但没有BOM，默认是BE，但Intel x86架构使用的是LE，所以很多文件用的是不带BOM的UTF-16LE。

In [178]:
u16le = 'El Niño'.encode('utf_16le')
print(u16le, type(u16le))
print(list(u16le))

b'E\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00' <class 'bytes'>
[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]


In [179]:
u16be = 'El Niño'.encode('utf_16be')
print(list(u16be))

[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]


## 4.5 处理文本文件
处理文本的最佳实践是Unicode三明治：
* 尽早把输入的字节序列解码为字符串(`bytes`->`str`)
* 100%处理字符串，处理过程中不解码/编码
* 尽晚把输出的字符串编码成字节序列(`str`->`bytes`)

Python的`open()`函数会在读取文件时作必要的解码，以文本模式写入文件时还会做必要的编码，所以调用`my_file.read()`方法得到的（已经解码过了）和传给`my_file.write()`方法的都是`str`对象。

如果依赖默认编码可能会出问题，以下示例OS X不会报错，因为其`open`的默认编码方式是utf-8。但需要在多台设备或者多种场合下运行的代码，一定不能依赖默认编码，始终明确传入encoding参数。

In [193]:
open('cafe.txt', 'w', encoding='utf8').write('café')
open('cafe.txt').read()

'café'

继续上例

In [195]:
fp = open('cafe.txt', 'w', encoding='utf8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf8'>

In [196]:
fp.write('café') # 返回写入的unicode字符数

4

In [197]:
fp.close()

In [198]:
import os
os.stat('cafe.txt').st_size # utf8中é是两个字节

5

In [200]:
fp2 = open('cafe.txt', 'rb')
fp2

<_io.BufferedReader name='cafe.txt'>

In [201]:
fp2.read()

b'caf\xc3\xa9'

> 除非想判断编码，否则不要再二进制模式中打开__文本文件__；即便如此，也应该使用Chardet，而不是重新发明轮子。常规代码应该只是用二进制模式打开二进制文件。

### 编码默认值：一团糟
编码的默认设置有许多来源。

最好不要依赖默认值。遵从Unicode三明治的建议，而且始终在程序中显式制定编码。

In [204]:
import sys, locale

In [211]:
expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty() # 在notebook中为False
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')
for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', value)

# my_file.encoding文本文件默认使用locale.getpreferredencoding
# isatty: 检测文件是否连接到一个终端设备

 locale.getpreferredencoding() -> UTF-8
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> UTF-8
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> UTF-8
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> UTF-8
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> UTF-8
      sys.getdefaultencoding() -> utf-8
   sys.getfilesystemencoding() -> utf-8


`locale.getpreferredencoding`返回的编码是最重要的：这是打开文本文件的编码，也是重定向到文件的`sys.stdin/stdout/stderr`的默认编码。

`sys.getfefaultencoidng()`用于Python在二进制数据和字符之间转换，这个设置用户不能修改。

`sys.getfilesystemencoding()`用于编解码文件名（不是文件内容），比如`open()`处理传入的字符串参数时。

## 4.6 为了正确比较而规范化Unicode字符串
该问题对于ASCII世界来说很简单，但对于Unicode来说很复杂。

因为Unicode有组合字符（变音符号和附加到前一个字符上的记号），打印时作为一个整体，所以字符串表示起来很复杂。
例如，'café'有两种表示方法，在之前的Python版本中，两种方式分别有4/5个码位。

In [216]:
s1 = 'café'
s2 = 'cafe\u0301' 
print(s1, s2)

café café


In [217]:
print(len(s1), len(s2))

5 5


In [218]:
s1 == s2

True

U+0301是COMBINING ACUTE ACCENT，加在e后面得到é。在Unicode标准中，以上两种序列是标准等价物（canonical equivalent），应用程序把它们视作相同的字符。但是Python（可能是原先的版本）看到的是不同的码位序列，因此判定不相等。

解决方案是`unicodedata.normalize`函数提供的Unicode规范化，这个函数的第一个参数有4个可选的值：
* `NFC`: Normalized Form C, 使用最少的码位构成等价的字符串。
* `NFD`: 把组合字符分解成基字符和单独的组合字符。
* `NFKC`/`NFKD`: K表示compatibality兼容性。因为Unicode的目标虽然是为各个字符提供规范的码位，但是为了兼容现有的标准，有些字符会出现多次。这两种形式会将各个兼容字符替换成一个或多个兼容分解的字符，有格式损失，但可以为搜索和索引提供便利的中间表述，注意不要用于持久存储。

西方键盘通常能输出组合字符，所以用户输入的文本默认是NFC形式。不过安全起见最好使用`normalize('NFC', user_text)`清洗文本。

In [267]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
len(s1), len(s2)

(5, 5)

In [221]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [223]:
len(normalize('NFD', s2)), len(normalize('NFD', s2))

(5, 5)

In [264]:
s1 == s2

True

In [225]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [226]:
normalize('NFD', s1) == normalize('NFC', s2)

False

使用NFC的时候，有些字符会被规范成另一个字符。例如电阻的单位欧姆Ω会被规范成希腊字母大写的欧米伽，两个字符在视觉上时一样的，但是比较时并不相等。

In [227]:
from unicodedata import normalize, name
ohm = '\u2126'
ohm_c = normalize('NFC', ohm)
print(ohm, ohm_c)

Ω Ω


In [229]:
name(ohm), name(ohm_c)

('OHM SIGN', 'GREEK CAPITAL LETTER OMEGA')

In [230]:
ohm == ohm_c

False

In [231]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

`NFKC`/`NFKD`在兼容字符上的实践。

比如希腊字母表中已有μ（码位是U+03BC），但Unicode还是添加了微符号μ（U+00B5），以便与latin1相互转换，因此微符号是兼容字符。

In [234]:
from unicodedata import normalize, name
half = '\u00BD'
half

'½'

In [235]:
normalize('NFKC', half) # 如果搜索1/2还能搜索到'½'，那是很好的，所以NFKC等是有用的

'1⁄2'

In [242]:
micro = '\u00B5'
micro_kc = normalize('NFKC', micro)
micro, micro_kc

('µ', 'μ')

In [243]:
ord(micro), ord(micro_kc)

(181, 956)

In [245]:
name(micro), name(micro_kc)

('MICRO SIGN', 'GREEK SMALL LETTER MU')

### 4.6.1 大小写折叠
为搜索和索引准备文本时很有用的一个操作，即把所有文本变成小写，再做其他转换，`str.casefold()`方法支持。

对于只包含latin1的文本，`casefold()`的结果和`lower()`一样，除了以下两个例外：

In [248]:
micro = 'µ'
micro_cf = micro.casefold()
print(micro, micro_cf)
print(name(micro), ',', name(micro_cf))

µ μ
MICRO SIGN , GREEK SMALL LETTER MU


In [257]:
import unicodedata
eszett = 'ß'
eszett_cf = eszett.casefold()
print(eszett, eszett_cf)
print(name(eszett))

ß ss
LATIN SMALL LETTER SHARP S


与Unicode相关的任何问题一样，大小写折叠复杂，有许多语言上的特殊情况，但是Python核心团队尽力提供了一种方案，能满足大多数用户的需求。

### 4.6.2 规范化文本匹配实用函数

对大多数应用来说，NFC是最好的规范方式。不去分大小写的比较应该使用`casefold()`。

In [258]:
# Utility functions for normalized Unicode string comparison
from unicodedata import normalize

# using NPC, case sensitive
def nfc_equal(s1, s2):
    return normalize('NFC', s1) == normalize('NFC', s2)

# using NFC with case folding
def fold_equal(s1, s2):
    return normalize('NFC', s1).casefold() == normalize('NFC', s2).casefold()

In [286]:
s1 = 'café'
s3 = 'cafe\u0301'
s1 == s3

True

In [287]:
nfc_equal(s1, s2)

True

In [289]:
fold_equal(s1, s2)

True

In [291]:
s3 = 'Straße'
s4 = 'Strasse'
s3 == s4

False

In [292]:
nfc_equal(s3, s4)

False

In [293]:
fold_equal(s3, s4) # 因为用了casefold

True

In [294]:
nfc_equal('A', 'a'), fold_equal('A', 'a')

(False, True)

### 4.6.3 极端“规范化”：去掉变音符号
Google搜索涉及很多技术，其一显然是忽略变音符号（如重音符、下加符等）。去掉变音符号不是正确的规范方式，但是现实生活中却很有用。

In [322]:
# 去掉全部变音符号的函数
import unicodedata
import string

def shave_marks(txt):
    """去掉全部变音符号"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved) 

In [323]:
s1 = 'café'
shave_marks(s1)

'cafe'

In [303]:
unicodedata.combining(normalize('NFC', 'é'))

0

In [324]:
unicodedata.combining(normalize('NFD', 'é')[0]), unicodedata.combining(normalize('NFD', 'é')[1]) 
# 0: no combining class
# 注意：NFD拆开后是基字符在前，组合字符在后

(0, 230)

上面的`shave_marks`函数做得太多了，通常去掉变音符号是为了让拉丁文本变成纯粹的ASCII，但是该函数还会修改希腊字母，但是去掉变音符号并不能让它们变成ASCII字符。

In [325]:
def shave_marks_latin(txt):
    """把拉丁基字符中的所有变音符号删除"""
    norm_txt = normalize('NFD', txt)
    latin_base = False
    keepers = []
    
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:
            continue # 不保留latin字母的组合字符
        keepers.append(c)
        # 如果不是组合字符，则为新的基字符
        if not unicodedata.combining(c):
            latin_base = c in string.ascii_letters
    return normalize('NFC', ''.join(keepers))

In [326]:
greek = 'ὦμέγα'
print(shave_marks(greek))
print(shave_marks_latin(greek))

ωμεγα
ὦμέγα


更彻底的规范化是把西文文本中的常见符号（如弯折号、长破折号、项目符号等）替换成ASCII中的对等字符。对应函数见书中，使用的是`str.maketrans()`和`translate()`函数。

注意以上标准化操作需要考虑到目标语言中变音符号的使用规则和转换后的用途等。

## 4.7 Unicode文本排序
Python比较任何类型的序列时，会一一比较序列内的元素，比如字符串比较的是码位。但是对于___非ASCII字符___，结果不尽如人意，主要涉及到变音符号对排序的影响。

Python中非ASCII字符的标准排序方式是使用`locale.strxfrm`函数，可以自己设置地区，这个设置是全局的。

In [ ]:
fruits = ['caju', 'atemoia', '', 'açaí']

In [327]:
# import locale
# locale.setlocale(locale.LC_COLLATE, 'zh_CN.UTF-8')
# fruits = []
# sorted(fruits, key=locale.strxfrm)

标准库的方案可用，但对不同操作系统的支持不够友好，还依赖区域设置，可以用PyPI中的PyUCA（Unicode Collation Algorithm）库。
#### 使用Unicode排序算法排序

#### 补充知识：字符串的一些方法

In [90]:
s = 'HeLLo'
s.casefold() # 全部转小写

'hello'

In [91]:
print('MyPy'.isidentifier()) # 是否是有效标识符，只含字母数字和下划线，不含空格，不以数字开头
print('__init__'.isidentifier())

True
True


In [92]:
print('\n'.isprintable()) # 换行
print('\t'.isprintable()) # 制表
print('\r'.isprintable()) # 回车
print('\\'.isprintable())
print('hello'.isprintable())

False
False
False
True
True


In [95]:
intab = 'aeiou'
outtab = '12345'
trantab = str.maketrans(intab, outtab)  # 制作翻译表
s = 'this is string example...wow!!!'
print (s.translate(trantab))

th3s 3s str3ng 2x1mpl2...w4w!!!


#### 补充知识：`isdigit()`, `isdecimal()`, `isnumeric()`
>`isdigit()`
* True: Unicode数字，全角数字（双字节），___byte数字（单字节）___
* False: 汉字数字，小数
* Error: 无 

>`isdecimal()`
* True: Unicode数字，全角数字（双字节）
* False: 汉字数字,小数
* Error: byte数字（单字节） 

>`isnumeric()`
* True: Unicode数字，全角数字（双字节），汉字数字 
* False: 小数 
* Error: byte数字（单字节）

亲测`isnumeric()`并不支持罗马数字。

In [84]:
num = "1"#unicode
print(num.isdigit()) # True
print(num.isdecimal()) # True
print(num.isnumeric()) # True
print(' ' * 15)
num = "1"# 全角
print(num.isdigit()) # True
print(num.isdecimal()) # True
print(num.isnumeric()) # True
print(' ' * 15)
num = b"1"# byte
print(num.isdigit()) # True
# print(num.isdecimal()) # AttributeError ‘bytes’ object has no attribute ‘isdecimal’
# print(num.isnumeric()) # AttributeError ‘bytes’ object has no attribute ‘isnumeric’
print(' ' * 15)
num = "四"# 汉字
print(num.isdigit()) # False
print(num.isdecimal()) # False
print(num.isnumeric()) # True

True
True
True
               
True
True
True
               
True
               
False
False
True


In [67]:
# isnumetic检查文本中所有字符是否都是数字，可以是其他进制，可以是其他语言，比如中文
a = "\u0030" #unicode for 0
b = "\u00B2" #unicode for ²
c = "10km2"
d = "-1"
e = "1.5"

print(a.isnumeric())
print(b.isnumeric())
print(c.isnumeric())
print(d.isnumeric())
print(e.isnumeric())

True
True
False
False
False


In [89]:
# 检查unicode对象中的所有字符是否都是十进制数，即全部属于0-9
print('abc'.isdecimal())
print('10'.isdecimal())

print("\u0030".isdecimal())
print("\u00B2".isdecimal())

False
True
True
False


In [171]:
int('l', base=16)

ValueError: invalid literal for int() with base 16: 'l'